In [50]:
#import required libraries
import math
import numpy as np
import pandas as pd
import re
import pyphonetics
import phonetics
import pydata_google_auth
import Monkey_Type_Detection as mtd
from arcgis.gis import GIS
from getpass import getpass
import skimpy
import wikipedia as wiki
import requests
from arcgis.geocoding import batch_geocode, Geocoder, get_geocoders, batch_geocode, geocode
from google_trans_new import google_translator

construct ACT Tree structure

In [2]:
class TreeNode:
    def __init__(self, data):
        self.data = data
        self.children = []
        self.parent = None

    def get_level(self):
        level = 0
        p = self.parent
        while p:
            level += 1
            p = p.parent

        return level

    def print_tree(self):
        spaces = ' ' * self.get_level() * 3
        prefix = spaces + "|__" if self.parent else ""
        print(prefix + str(self.data))
        if self.children:
            for child in self.children:
                child.print_tree()

    def add_child(self, child):
        child.parent = self
        self.children.append(child)
###CLASS Definition ends here###
# import re
# re.seacrh("", element)

In [60]:
def get_file_ext(file_name):
    file_ext=file_name[len(file_name)-5:].rpartition('.')[1]
    return file_ext

def read_file(file_name,file_ext):
    #extract file name excluding the file extension -
    #add other possible file formats for data source here.
    file_name_without_ext=file_name.rpartition('.')[0]
    df=pd.DataFrame()
    if file_ext=='csv':
        df=pd.read_csv(file_name)
    elif file_ext=='pq':
        df=pd.read_parquet(file_name)
    elif file_ext=='pkl':
        df=pd.read_pickle(file_name)
    elif file_ext=='json':
        df=pd.read_json(file_name)

    else:
        raise Exception("Invalid file format...", file_ext)
    return df

def get_postal_pincode_data():
    
    from skimpy import clean_columns



    postal_pincode_data=pd.read_csv('postal_pincode_data.csv')
    messy_df = postal_pincode_data
    clean_df = clean_columns(messy_df)
    clean_df.columns.tolist()

def regex_match(pincode,pattern):
    if re.match(pincode, pattern):
        return True
    else:
        return False



def check_pincode_correct(pincode, address):
    pincode=str(pincode) #convert the input pincode - into string format, if not already - the first digit in pincode is zero - it will be removed, and pincode - will be read as 5 digits
    #check if it has 6 digits
    if len(pincode)<6:
        raise Exception("Invalid pincode - pincode contains less than 6 digits")
    elif len(pincode)>6:
        raise Exception("Pincode cannot have more than 6 digits...")
    elif pincode[0]=='0':
        raise Exception("First digit of pincode cannot be zero...")


    pincode_pattern=r"^[1-9]{1}[0-9]{2}\\s{0,1}[0-9]{3}$"
    pincode_pattern_valid=regex_match(pincode_pattern,pincode)
    if (pincode_pattern_valid==False):
        raise Exception("Pincode Format not valid...")
    else: #entered pincode format is valid
        #now check if the entered pincode present in promise or postal database -
        postal_pincode_df=pd.read_csv('postal_pincode_data.csv')
        #convert both postal pincode and promise pincode list - to str types - to append both lists
        postal_pincode_df['pincode']=postal_pincode_df['pincode'].astype(str)
        postal_pincode_list=list(postal_pincode_df['pincode'].drop_duplicates().to_numpy())
        print("len of postal_pincode_list: ",len(postal_pincode_list))

        promise_pincode_df=pd.read_csv('promise_pincode_data.csv')
        promise_pincode_df['pincode']=promise_pincode_df['pincode'].astype(str)
        promise_pincode_list=list(promise_pincode_df['pincode'].drop_duplicates().to_numpy())
        print("len of promise_pincode_list: ",len(promise_pincode_list))
        #get combined list of two pincodes -
        pincode_list=list(set(postal_pincode_list)|set(promise_pincode_list))
        print("len of pincode list: ",len(pincode_list))



        #check if pincode present in the overall pincode list -
        if pincode not in pincode_list:
            raise Exception("Pincode not found in database...")
        else:
            #now if the format of the entered pincode is valid and if it is also found in the overall pincode database...
            #Next step is to see if the pincode matches - the entered address
            #Two ways to check -
            # if the city matches pincode
            # if the locality or other smaller - division - mapped to a pincode - based on postal pincode data
                        """
                        Postal data - attributes -
                        officename	pincode	officeType	Deliverystatus	divisionname regionname	circlename	Taluk	Districtname	statename	Telephone	Related Suboffice	Related Headoffice	longitude	latitude

                        """
        pincode_expected_address_component_df=postal_pincode_df.loc[postal_pincode_df['pincode']==pincode,['officename','officeType','divisionname','regionname','circlename','Taluk','Districtname','statename','Related Suboffice','Related Headoffice','longitude','latitude']]
        postal_data_cols=['officename','officeType','divisionname','regionname','circlename','Taluk','Districtname','statename','Related Suboffice','Related Headoffice','longitude','latitude']
        for col in postal_data_cols:
            #convert dtype to str - for all the address_dataset part strings
            pincode_expected_address_component_df.loc[pincode_expected_address_component_df[col].isnull()==True,col]="None"
            pincode_expected_address_component_df[col]=pincode_expected_address_component_df[col].astype(str)
            #convert the address_dataset substr - to lower
            pincode_expected_address_component_df[col]=pincode_expected_address_component_df[col].str.lower()
            #get pincode's correspondong - address string - address
            pincode_address_mismatch=pincode_address_match_similarity(pincode,address,pincode_expected_address_component_df)










def pincode_address_match_similarity(pincode, address, expected_address):
    #check for expected address' similarity with actual address given for a pincode -

#to map address to the type - for eg. if an address corresponds to an apartment or individual building
        """
        1. solution 1 - use Google Maps API to see if the extracted door number of a specific street name corresponds to an apartment or a house - use Selenium - to extract the address details - corresponding to the search - if we need to avoid a geolocation API

        2. solution 2 - analyze the deliveries history for the same address (door number + street name) and if the total number of deliveries is significantly higher than the average, it should be an indicator that it refers to an apartment or office space, if it significantly lower it may point to it being a house
        """
def set_typology_feature_for_address(address):
    pass

#to classify an address - residential or a commercial/industrial area

"""
1. infer from address corresponding details - using geolocation API/extract addition location info - using selenium from maps.google.com - used for tagging area type -
2. analyze - the neigbouring addresses - if they are tagged - by area type and infer - area type - based on the nearest address info we have
"""
def set_area_feature_for_address(address):
    pass

def locality_set_for_given_pincode(pincode):
    """
    refer to ACT db - to get the range of localities a pincode is mapped to - along with
    only use delivered address examples without any address issue or pincode address mismatch issue identified - also
    use - courier partner's lat long and address lat long - from shopify or geolocation api mapping (used for training data)
    if the distance between the two lat long coordinates - less than a threshold -
    """
    pass




#clean data - remove missing, incorrect pincodes,duplicate pincodes
#get pincode, taluk mapping
def geolocation_mapping_fwd(x): #address to lat long mapping
    #using geopy, using arcgis, using position stack
    """ http://api.positionstack.com/v1/forward
        ? access_key ="""
    import requests
    #try getting location data from google maps
    response = requests.get('https://google.com/')
    print(response)
    """<Response [200]>
        & query = 1600 Pennsylvania Ave NW, Washington DC"""

def geolocation_mapping_reverse(x): #lat long to address -> address entity mappingdef composite_function(f, g):
        # return lambda x : f(g(x))
        pass




###



def pincode_validation(pincode):
    pincode_format_valid=check_pincode_format(pincode)

def address_raw_data(data_source_file):
    #identify the type of file -
    #get file extention -
    file_ext=data_source_file.get_file_ext()
    #get address data - from different sources -
    # 1. Shipment History
    # 2. Shopify and Woocom Orders data - both swift/non-swift fulfilled - shipping and billing address, pickup addresses
    # 3. External Data Sources -
    #   a. data gov - websites - indian address data sources (need additional sources - MapMyIndia,
    #   b. OSM Indian Address data
    #   c. Require - list of zone - ward, street level info - whenever available for a city
    #   c. Amazon - lat long mapping for the delivered addresses - uses GPS Coordinates (?)
    #   d. Delhivery - lat long mapping for the delivered addresses - uses GPS Coordinates

    address_data=read_file_name(data_source_file,file_ext)
    #if valid data source exist in the given data source -
    if address_data.empty==False:
        #extract the required fields -
        # use std naming for address data attributes - across all data sources
        # address_string, pincode, city, state
        # use promise data for mapping - pincode - to city, state
        pass
    pass


def isEnglish(s):
    return s.isascii()

def get_soundex(name):
        """Get the soundex code for the string"""
        name = name.upper()

        soundex = ""
        soundex += name[0]

        dictionary = {"BFPV": "1", "CGJKQSXZ":"2", "DT":"3", "L":"4", "MN":"5", "R":"6", "AEIOUHWY":"."}

        for char in name[1:]:
            for key in dictionary.keys():
                if char in key:
                    code = dictionary[key]
                    if code != soundex[-1]:
                        soundex += code

        soundex = soundex.replace(".", "")
        soundex = soundex[:4].ljust(4, "0")

        return soundex


def text_preprocessing(text):
    #standard stemming, lemmatization - should be avoided in case of address data - as the exact word is required
    #remove special characters, and convert address string to lowercase
    # x = re.search("^A-Za-z0-9", txt)
    ptext=''
    #remove non-ascii characters
    for x in text:
        if isEnglish(x)==True:
            ptext=ptext+x

    preprocessed_text=ptext
    ptext=''

    for x in preprocessed_text:
        if x.isalnum()==True:
            ptext=ptext+x
        #replace special characters with space
        elif x.isalnum()==False:
            ptext=ptext+' '

    preprocessed_text=ptext
    #convert string to lower
    preprocessed_text=preprocessed_text.lower()

    #spelling correction - for city, state,


    #phonetic similarity
def soundex_similarity(text,text_list):
    from pyphonetics import Soundex
    soundex = Soundex()
    for t in text_list:
        replace_text_with=t
        print("t: ",t)
        print("t type: ",type(t))
        text=text.encode('utf-8').decode('utf-8')
        t=t.encode('utf-8').decode('utf-8')
        text_similar=soundex.sounds_like(text,t)
        # text_similarity_score=soundex.phonetics(text,t)
        if text_similar:
            replace_text_with=t
            break
    return replace_text_with
def metaphone_similarity(text,text_list):
    from pyphonetics import Metaphone
    metaphone = Metaphone()
    for t in text_list:
        replace_text_with=t
        text=text.encode('utf-8').decode('utf-8')
        t=t.encode('utf-8').decode('utf-8')
        text_similar=metaphone.sounds_like(text,t)
        # text_similarity_score=metaphone.phonetics(text,t)
        if text_similar:
            replace_text_with=t
            break
    return replace_text_with

def refined_soundex_similarity(text,text_list):
    from pyphonetics import RefinedSoundex
    refined_soundex = RefinedSoundex()
    for t in text_list:
        replace_text_with=t
        text=text.encode('utf-8').decode('utf-8')
        t=t.encode('utf-8').decode('utf-8')
        text_similar=refined_soundex.sounds_like(text,t)
        # text_similarity_score=refined_soundex.phonetics(text,t)

        if text_similar:
            replace_text_with=t
            break
    return replace_text_with

def fuzzy_soundex_similarity(text,text_list):
    from pyphonetics import FuzzySoundex
    fuzzy_soundex = FuzzySoundex()
    for t in text_list:
        replace_text_with=t
        text=text.encode('utf-8').decode('utf-8')
        t=t.encode('utf-8').decode('utf-8')
        text_similar=fuzzy_soundex.sounds_like(text,t)
        # text_similarity_score=fuzzy_soundex.phonetics(text,t)

        if text_similar:
            replace_text_with=t
            break
    return replace_text_with

def levenshtein_similarity(text,text_list):
    from pyphonetics import RefinedSoundex
    levenshtein_similarity = RefinedSoundex()
    for t in text_list:
        replace_text_with=t
        text=text.encode('utf-8').decode('utf-8')
        print("t: ",t)
        print("t type: ",type(t))
        t=t.encode('utf-8').decode('utf-8')
        text_similar=levenshtein_similarity.distance(text,t,metric='levenshtein')
        # text_similarity_score=levenshtein_similarity.phonetics(text,t)
        if text_similar==0:
            replace_text_with=t
            break
    return replace_text_with


def hamming_similarity(text,text_list):
    from pyphonetics import RefinedSoundex
    levenshtein_similarity = RefinedSoundex()

    for t in text_list:
        replace_text_with=t
        text=text.encode('utf-8').decode('utf-8')
        t=t.encode('utf-8').decode('utf-8')
        text_similar=levenshtein_similarity.distance(text,t,metric='hamming')
        # text_similarity_score=levenshtein_similarity.phonetics(text,t)
        if text_similar==0:
            replace_text_with=t
            break
    return replace_text_with
def phonetic_similarity_city_and_state(city, state):
    #get the similarity score using each type of phonetic similarity matching
    #using soundex algorithm
    promise_data=pd.read_parquet('promise_state_name.pq')
    #convert all strings in the df - to lower
    promise_data['city']=promise_data['city'].apply(lambda x: str.lower(x))
    promise_data['stateName']=promise_data['stateName'].apply(lambda x: str.lower(x))
    city_list=list(promise_data.loc[promise_data['city'].isnull()==False,'city'].drop_duplicates().to_numpy())
    state_list=list(promise_data.loc[promise_data['stateName'].isnull()==False,'stateName'].drop_duplicates().to_numpy())
    replace_state_name_with=state
    replace_city_name_with=city
    if replace_state_name_with not in state_list:
        # try mapping a state name - using soundex similarity
            replace_state_name_with=levenshtein_similarity(state,state_list)
            if replace_state_name_with not in state_list:
                replace_state_name_with=hamming_similarity(state,state_list)
                if replace_state_name_with not in state_list:
                    replace_state_name_with=soundex_similarity(state,state_list)
                    if replace_state_name_with not in state_list:
                        replace_state_name_with=metaphone_similarity(state,state_list)
                        if replace_state_name_with not in state_list:
                            replace_state_name_with=refined_soundex_similarity(state,state_list)
                            if replace_state_name_with not in state_list:
                                replace_state_name_with=fuzzy_soundex_similarity(state,state_list)
                                if replace_state_name_with not in state_list:
                                                print("no exact state name match found....")

    if replace_city_name_with not in city_list:
        # try mapping a city name - using soundex similarity

            replace_city_name_with=levenshtein_similarity(city,city_list)
            if replace_city_name_with not in city_list:
                replace_city_name_with=hamming_similarity(city,city_list)
                if replace_city_name_with not in city_list:
                    replace_city_name_with=soundex_similarity(city,city_list)
                    if replace_city_name_with not in city_list:
                        replace_city_name_with=metaphone_similarity(city,city_list)
                        if replace_city_name_with not in city_list:
                            replace_city_name_with=refined_soundex_similarity(city,city_list)
                            if replace_city_name_with not in city_list:
                                replace_city_name_with=fuzzy_soundex_similarity(city,city_list)
                                if replace_city_name_with not in city_list:
                                    replace_city_name_with=levenshtein_similarity(city,city_list)
                                    if replace_city_name_with not in city_list:
                                        replace_city_name_with=hamming_similarity(city,city_list)
                                        if replace_city_name_with not in city_list:
                                            print("no exact city name match found....")
    return replace_city_name_with, replace_state_name_with

# def spell_corrector
def translate_to_english(text):

    translator = google_translator()

    sentence = text
    translate_text = translator.translate(sentence,lang_tgt='en')
    return translate_text


In [15]:
# root = TreeNode("India")
address_tree=TreeNode("India")
def build_address_tree():
    address_db=pd.read_csv('dataset_promise_merge.csv')
    # 'pincode', 'city', 'state', 'stateName', 'tier'
    # 'drop_location_pincode', 'drop_location_address'
    address_db_short=address_db[['drop_location_address','drop_location_pincode','city','tier','stateName']]
    print("address db: ")
    print(address_db_short)
    print("len of decision tree: ",len(address_db))
    print("address columns : ",address_db.columns)
    # global root
    root = TreeNode("India") #root node
    promise_state_name=pd.read_parquet('promise_state_name.pq')
    state_list=list(promise_state_name['stateName'].drop_duplicates().to_numpy())
    print("state_list: ",state_list)
    print("******************************")
    city_list=list(promise_state_name['city'].drop_duplicates().to_numpy())
    print("city_list: ",city_list)
    #converting the city and state list to lower - 
    state_list_lower=[]
    for state in state_list:
        state=state.lower()
        state_list_lower.append(state)
    city_list_lower=[]
    for city in city_list:
        city=city.lower()
        city_list_lower.append(city)
    print("city list length : ",len(city_list_lower))

    print("state list length : ",len(state_list_lower))

    promise_state_name['pincode']=promise_state_name['pincode'].astype(float)
    promise_state_name.loc[promise_state_name['pincode'].isnull()==False,'district_pin']=(promise_state_name['pincode']/1000).astype(int)
    promise_state_name
    #add states as child nodes for root node - pointing to country
    for state in state_list:
#         dpin_list_for_state=list(promise_state_name.loc[promise_state_name['stateName']==state,'district_pin'].drop_duplicates().to_numpy())
        
        #add district pins as child nodes for every state node
        state_node=TreeNode(str(state))
        """for dpin in dpin_list_for_state:
                city_for_dpin=list(promise_state_name.loc[promise_state_name['district_pin']==dpin,'city'].drop_duplicates().to_numpy())
                dpin_node=TreeNode(str(dpin))"""
        city_for_state=list(promise_state_name.loc[promise_state_name['stateName']==state,'city'].drop_duplicates().to_numpy())
        for city in city_for_state:  
                    #assuming a city will have only pincodes - with same starting 3 digits, i.e a city will
                    #have a single district pin - 3 digit no.)
                    #add all those cities as child nodes within district node
                    city_node=TreeNode(str(city)) 
                    #create a city node - which can be further extended for granular location levels
                    
#                     dpin_node.add_child(city_node) #add city to a district pin as a child node.
                    #for every city - has many locality, larger neighborhoods, followed by a pincode level grouping
                    #for a city - find the list of locality, for a locality - get a list of sub localities
                    
                    state_node.add_child(city_node)
        root.add_child(state_node)
    
    address_tree=root
    root.print_tree()
    return address_tree


In [16]:
address_tree=build_address_tree()

# dataset_promise_merge.csv

address db: 
                                     drop_location_address  \
0        ,A405, Aditya jalakrishna, Deepthisri nagar, M...   
1        null,H no 12 gokuldham campus Near vidhya saga...   
2        Sector 77 C-UG01 Civitech Sampriti  Noida Utta...   
3        ,Flat no 902, Alya, Kabra Galaxy Star 3, Brahm...   
4        ,520-A, The Magnolias, DLF Golf Links, Golf Co...   
...                                                    ...   
1218604  Plot no 1000, flat no 304 Rainbow ResidencyPra...   
1218605  PNo:5-442, 1st Floor, Gokul plots, Kphb 9th Ph...   
1218606  a/501 5th floor columbia palava dombivali east...   
1218607  Meghana paradise zion block 203, Nagavarapalya...   
1218608  BPTP Park Center, Sector 30, Jalvayu Vihar, Ne...   

         drop_location_pincode           city  tier       stateName  
0                       500050      hyderabad   1.0       Telangana  
1                       462030         bhopal   2.0  Madhya Pradesh  
2                       201301  

In [17]:
address_tree.print_tree()

India
   |__Maharashtra
      |__nashik
      |__banda mh
      |__waluj
      |__sillod
      |__saidapura satara
      |__pirangut
      |__pune
      |__nasrapur mh
      |__malashiras
      |__indapur
      |__mandrup
      |__kem
      |__bhose
      |__shirur anantpal
      |__terkheda
      |__anala
      |__aundha nagnath
      |__andur
      |__murum
      |__newasa
      |__kada
      |__bambavade
      |__bhavaninagar
      |__lonand
      |__dahiwadi
      |__mhaswad
      |__jawaharnagar
      |__hupari
      |__talere
      |__walwa
      |__kavathemahankal
      |__kalyan
      |__talegaon dhige
      |__wadwani
      |__rahata
      |__lasur
      |__fardapur
      |__ambad
      |__pachod
      |__gevrai
      |__sonpeth
      |__purna
      |__mudkhed
      |__kalamnuri
      |__umri
      |__mumbai
      |__thane
      |__navi mumbai
      |__greater thane
      |__jasai
      |__boisar
      |__vasai
      |__kudus
      |__palghar
      |__dahanu
      |__mahad
   

In [2]:
#consolidate all data to one - use multiprocessing to load data from various sources
open_street_address_db=pd.read_pickle('formatted_open_street_address_db.pkl')
woocom_data=pd.read_pickle('woocom.pkl')
shopify_data=pd.read_pickle('shopify_data.pkl')
formatted_open_street_address_db=pd.read_pickle('formatted_open_street_address_db.pkl')
dataset_promise_merge=pd.read_csv('dataset_promise_merge.csv')

In [6]:
pd.set_option('display.max_columns',None)

In [7]:
shopify_data.head()

,id,unique_shipment_id,Shipping_Name,Shipping_Phone_Number,user_agent,customer_browser_ip,seller_id,orderCtime,shipping_address_zip,shipping_address_city,customer_state,customer_name,customer_accepts_marketing,customer_verified_email,customer_total_spent,customer_orders_count,customer_phone,customer_email,customer_last_order_id,last_order_name,shipping_address_latitude,shipping_address_longitude,shipping_address_address1,shipping_address_address2,shipping_address,default_address_name,default_address_zip,default_address_address1,default_address_address2,billing_address,shop,confirmed,order_status_url,cancelled_at,updated_at,gateway,financial_status,landing_site,quantity,referring_site,source_name,line_item_title,line_item_qty,line_item_per_qty_price,line_item_per_qty_wt,total_price,total_weight
0,3074934407325,None,Tanya Arora,7310262626,"Mozilla/5.0 (iPhone; CPU iPhone OS 13_6_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 Instagram 174.0.0.21.119 (iPhone8,1; iOS 13_6_1; en_IN; en-IN; scale=2.00; 750x1334; 271836321)",223.236.160.168,auth0|60406ec7366ce3006919b8a4,2021-02-14T11:26:13Z,226004,Lucknow,disabled,None,false,true,499.00,1,+917310262626,None,3.074934407325E12,#1006,None,None,19 datia house khurshed bagh near pukar factory,"Ganeshganj, lucknow","Ganeshganj, lucknow, 19 datia house khurshed bagh near pukar factory, Lucknow",None,226004,19 datia house khurshed bagh near pukar factory,"Ganeshganj, lucknow","Ganeshganj, lucknow, 19 datia house khurshed bagh near pukar factory, Lucknow",jinglegifts1.myshopify.com,true,https://jinglegifts.com/53584560285/orders/f0d1dbfb9fcac6c4effb663b2be8247c/authenticate?key=bd2a872e8ec675d033424b423c04d790,None,2021-02-15T10:30:21+05:30,Cash on Delivery (COD),pending,/products/mahakaal-shiva-om-bracelet,0,,web,Shiv Shakti Bracelet,1,499.00,0,499.00,0
1,2965863497886,500220B2049401,Arshan Randhawa,+91 99888 47449,"Mozilla/5.0 (Linux; Android 10; SM-A515F Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/87.0.4280.101 Mobile Safari/537.36 Instagram 168.0.0.40.355 Android (29/10; 420dpi; 1080x2186; samsung; SM-A515F; a51; exynos9611; en_GB; 261079769)",101.0.41.130,auth0|5f57a8f14c5c7a006ea12985,2020-12-12T06:44:35Z,160001,Chandigarh,disabled,None,true,true,190.00,1,None,arshan.randhawa@gmail.com,2.965863497886E12,#1336,None,None,"#75, sector 2",,", #75, sector 2, Chandigarh",None,160001,"#75, sector 2",,", #75, sector 2, Chandigarh",epicurry.myshopify.com,true,https://www.epicurry.com/47268495518/orders/a89b0c4197c5c0a1c0b0097dd4447af5/authenticate?key=611a79dc405f0cc094fa4bce99249aab,None,2020-12-12T12:14:37+05:30,payu_in_paisa,paid,/?utm_source=facebook&utm_medium=ads&utm_campaign=Masala_Lookalike_Nasi_Goreng&fbclid=PAAaYa8jblHChM3LQoWQiKn1hrXC7LKuhWRVrPZHZRn8bpI6qRq9inXC2gvII,1,http://instagram.com/,web,Nasi Goreng with Chicken Satay Masala-Kit,1,150.00,110,190.00,110
2,2289144856635,500079B5904734,shyam bhagat,98349 30489,"Mozilla/5.0 (Linux; Android 8.1.0; CPH1859 Build/O11019; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/71.0.3578.99 Mobile Safari/537.36 [FB_IAB/FB4A;FBAV/270.1.0.66.127;]",157.47.54.109,auth0|5e6a18f52cb0aa0c89f58a03,2020-05-25T09:25:08Z,413515,Ahmedpur,disabled,None,true,true,1499.00,1,None,bhagatshyam80@gmail.com,2.289144856635E12,#1591,18.7033225,76.9365479,Patil Galli Masjid-E-Markas Dastgir Colony,near nootan marathi vidyalaye,"near nootan marathi vidyalaye, Patil Galli Masjid-E-Markas Dastgir Colony, Ahmedpur",None,413515,Patil Galli Masjid-E-Markas Dastgir Colony,,", Patil Galli Masjid-E-Markas Dastgir Colony, Ahmedpur",flaunt-the-store.myshopify.com,true,https://flauntstore.com/33463205947/orders/b39158aafd69e5a9e0d3027f0d20a330/authenticate?key=1a9f0e29966168e01a64e251a59db8c8,None,2020-05-27T16:51:30+05:30,cash_on_delivery,pending,/wallets/checkouts.json,1,https://flauntstore.com/products/neck-breezer,web,Neck Breezer™,1,1499.00,500,1499.00,500
3,3720478851234,500354K9772724,Amit R

In [23]:
password=getpass()
gis=GIS("https://www.arcgis.com","jarvisvb",password)

In [34]:
type(gis)

arcgis.gis.GIS

In [39]:
from arcgis.geocoding import batch_geocode, Geocoder, get_geocoders, batch_geocode, geocode

In [25]:
help(batch_geocode)

Help on function batch_geocode in module arcgis.geocoding._functions:

batch_geocode(addresses, source_country=None, category=None, out_sr=None, geocoder=None, as_featureset=False, match_out_of_range=True, location_type='street', search_extent=None, lang_code='EN', preferred_label_values=None)
    The ``batch_geocode`` function geocodes an entire list of addresses.
    
    .. note::
        Geocoding many addresses at once is also known as bulk geocoding.
    
    =========================     ================================================================
    **Argument**                  **Description**
    -------------------------     ----------------------------------------------------------------
    addresses                     Required list of strings or dictionaries.
                                  A list of addresses to be geocoded.
                                  For passing in the location name as a single line of text -
                                  single field

In [43]:
items = gis.content.search('Geocoder', 'geocoding service', max_items=3)
items

[]

In [40]:
geocode('UrbanVault 761, HSR Layout, Sector 3, Bangalore, 560102')

[{'address': 'Sector 3, HSR Layout, Bangalore, Bengaluru, Karnataka',
  'location': {'x': 77.64170000000007, 'y': 12.910750000000064},
  'score': 87.55,
  'attributes': {'Loc_name': 'World',
   'Status': 'T',
   'Score': 87.55,
   'Match_addr': 'Sector 3, HSR Layout, Bangalore, Bengaluru, Karnataka',
   'LongLabel': 'Sector 3, HSR Layout, Bangalore, Bengaluru, Karnataka, IND',
   'ShortLabel': 'Sector 3',
   'Addr_type': 'Locality',
   'Type': 'Zone',
   'PlaceName': 'Sector 3',
   'Place_addr': 'HSR Layout, Bangalore, Bengaluru, Karnataka',
   'Phone': '',
   'URL': '',
   'Rank': 15,
   'AddBldg': '',
   'AddNum': '',
   'AddNumFrom': '',
   'AddNumTo': '',
   'AddRange': '',
   'Side': '',
   'StPreDir': '',
   'StPreType': '',
   'StName': '',
   'StType': '',
   'StDir': '',
   'BldgType': '',
   'BldgName': '',
   'LevelType': '',
   'LevelName': '',
   'UnitType': '',
   'UnitName': '',
   'SubAddr': '',
   'StAddr': '',
   'Block': '',
   'Sector': 'Sector 3',
   'Nbrhd': '',
 

In [44]:
import wikipedia as wiki
import requests
from arcgis.geocoding import batch_geocode, Geocoder, get_geocoders, batch_geocode, geocode

In [54]:
#shopify data - preprocess address strings
shopify_data.groupby('shipping_address_zip',dropna=False).size()

shipping_address_zip
              295
\t 811202       2
\t110006        4
\t110018        3
\t110094        3
             ... 
२०६००१          2
४१०५०६          1
४१५३०५          3
੧੪੭੧੦੧          1
NaN          8414
Length: 24257, dtype: int64

In [58]:
google_translator("४१०५०६")

In [55]:
import detect_delimiter

In [74]:
def split_by_delimiters(text):
    split_string = re.split(r',|!|;|-|_| |\t|\n', text)
    print(text)
    print(split_string)
    return split_string #return lexical units - split by various delimiters

#test run
text = """hi! my na-me is ni_k, we\tcome; to datagy
good
bad"""
split_by_delimiters(text)

hi! my na-me is ni_k, we	come; to datagy
good
bad
['hi', '', 'my', 'na', 'me', 'is', 'ni', 'k', '', 'we', 'come', '', 'to', 'datagy', 'good', 'bad']


['hi',
 '',
 'my',
 'na',
 'me',
 'is',
 'ni',
 'k',
 '',
 'we',
 'come',
 '',
 'to',
 'datagy',
 'good',
 'bad']

In [ ]:
import swifter
key_freq_df=pd.DataFrame(columns={'key','frequency'})
def tag_split(x):
    #get a unique set of keys and their frequency - to add cols to df
    key_freq_row=pd.DataFrame(columns={'key','frequency'})
    key_values=list(x.keys())
    global key_freq_df
    for key in key_values:     
        if key_freq_df.loc[key_freq_df['key']==key].empty==True: 
            # if the key df - doesn't have this particular key - add this key to list
            #add this as a new key - in a new row
            key_freq_row[['key','frequency']]=[[key,1]]
            #add this key to the total list of keys
            
            key_freq_df=pd.concat([key_freq_df,key_freq_row])
            key_freq_df=key_freq_df.reset_index()
            del key_freq_df['index']
        else: #if a key of the same name is already present
            #increment the frequency by 1 of a particular key in the key df 
            key_freq_df.loc[key_freq_df['key']==key,'frequency']=key_freq_df['frequency']+1
open_street_address_db['tags'].swifter.apply(lambda x: tag_split(x))
# tag_split(open_street_address_db['tags'])

In [76]:
a=[]
a.append('1')
print(a)
a.append('2')
print(a)

['1']
['1', '2']


In [77]:
"hulahoop".find("hoo")

4

In [80]:
list("apple")

['a', 'p', 'p', 'l', 'e']

In [ ]:
def get_nearest_delim(word, text):
    comma_text_split=text.split(',')
    word_start_index=text.find(word)
    word_end_index=word_start_index+len(word)
    word_before_char=[]
    word_before_text=
    if word not in comma_text_split:
        return 0
    elif substr_before_text==text[(word_start_index-1)]:word_end_index]


In [ ]:
word_comma_freq=pd.DataFrame(columns={'word','comma_before_freq','comma_after_freq'})
all_words_comma_freq_track=pd.DataFrame(columns={'word','comma_before_freq','comma_after_freq'})
all_words_list=[]
def insert_comma_by_frequency(text):
    text_list=split_by_delimiters(text) #split by all the delimiters - to get individual words
    for word in text_list:
        if word not in all_words_list:
            all_words_list.append(word) #adding the newly encountered word into large address word corpus
            word_comma_freq['word']=word
            word_comma_freq['comma_before_freq']=0
            word_comma_freq['comma_after_freq']=0
    



    

            







        

In [ ]:
def rule_based_custom_ner(text):
    #identify some entities - to assign rule based tags
    #for eg. 
    #if house no, no., building no., etc and any number followed by comma - usually points to building name/building name within the street
    #street - based on the text that occur before street, road, lane, etc. - checking for other synonymns
    #colony, nagar, layout, - locality
    #neighbourhood - wiki scrape
    # district -  wiki scrape
    address_with_entities=dict()
    return address_with_entities

In [ ]:
def shopify_data_preprocess(df):
    #two address lines - 
    df.loc[df['shipping_address_address1'].isnull()==False,'shipping_address_address1']=df['shipping_address_address1'].str.lower()
    df.loc[df['shipping_address_address2'].isnull()==False,'shipping_address_address2']=df['shipping_address_address2'].str.lower()
    df.loc[df['shipping_address_address1'].isnull()==True,'shipping_address_address1']='' #set empty address
    df.loc[df['shipping_address_address2'].isnull()==True,'shipping_address_address2']=''
    #get overall address
    df.loc[df['shipping_address_address1'].isnull()==False,'whole_shipping_address']=df['shipping_address_address1']+','+df['shipping_address_address2']

    #check for invalid address - frequency
    df.rename(columns={'whole_shipping_address':'address_string','shipping_address_city':'drop_location_city','shipping_address_zip':'drop_location_pincode'},inplace=True)
    df.loc[df['shipping_address_city'].isnull()==False,'shipping_address_city']=df['shipping_address_city'].str.lower()
    

In [ ]:
def delimiter_insert(address):
    x[]

In [46]:
wiki.summary('Villivakkam')

'Villivakkam is a neighbourhood of Chennai, Tamil Nadu, India. It is served by the Villivakkam railway station on the Chennai Central–Arakkonam suburban section.  Villivakkam is the largest state assembly constituency in Tamil Nadu in terms of number of voters.'

In [ ]:
#custom NER 
def location_entity_mapping(x):
    

In [30]:
addresses=['UrbanVault 761, HSR Layout, Sector 3, Bangalore, 560102', 'Whatfix Towers, HSR Layout, Sector 3, 560102']

In [37]:
batch_geocode(addresses) #token='AAPK9e4568a156854a66a2ee504314e804edvqBIaWV8yNcDyAg3iNhvblf2KdcY6Wper9FQn77xLO9vQA9YezDQVAk1fmOKU53J')

Exception: Token required but not passed in the request.
Token required.
(Error Code: 499)

In [ ]:
item = gis.content.get("")


In [17]:

password=getpass()
gis=GIS("https://arcgis.com","jarvisvb",password)
# gis = GIS('home')
# gis?
# gis = GIS()
# gis.map('Berlin')

Exception: A general error occurred: Invalid username or password.

In [20]:
gis=GIS("http://www.arcgis.com","jarvisvb",password)

Exception: A general error occurred: Too many invalid attempts. Please try again later.

In [14]:
gis.map("India")

MapView(layout=Layout(height='400px', width='100%'))

In [ ]:
import wikipedia
# def wiki_scrape_locality_tag():
#read shopify data - to get address strings

